In [1]:
from spacy.tokens import DocBin
import spacy
import json
import os
from tqdm import tqdm
import random

C:\Users\vedan\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\vedan\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\vedan\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
def load_data():
    training_data = {}
    for item in os.listdir("datasets"):
        if ".json" in item:
            temp = json.load(open(f"./datasets/{item}", encoding="utf-8"))
            if training_data == {}:
                training_data = temp
            else:
                training_data["annotations"].extend(temp["annotations"])
    
    # convert all texts to lowercase
    for i in range(len(training_data["annotations"])):
        training_data["annotations"][i][0] = training_data["annotations"][i][0].lower()

    return training_data

In [3]:
train_data = load_data()

In [4]:
# nlp = spacy.blank("en")
nlp = spacy.load("en_core_web_sm")
def create_traning(TRAIN_DATA, split=0.8):
    db = DocBin()
    db_dev = DocBin()
    length = len(TRAIN_DATA)
    for i, (text, annot) in tqdm(enumerate(TRAIN_DATA)):
        doc = nlp.make_doc(text)
        ents = []
        for start, end, label in annot["entities"]:
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span is None:
                print("Skipping")
            else:
                ents.append(span)
        
        doc.ents = ents

        if i < length * split:
            db.add(doc)
        else:
            db_dev.add(doc)

    
    return db, db_dev

C:\Users\vedan\AppData\Local\Programs\Python\Python38\lib\site-packages\spacy\util.py:833: UserWarning: [W095] Model 'en_core_web_sm' (3.0.0) was trained with spaCy v3.0 and may not be 100% compatible with the current version (3.2.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [5]:
db, db_dev = create_traning(train_data["annotations"])

51it [00:00, 120.45it/s]


In [6]:
db.to_disk("./training_dataset/training.spacy")
db_dev.to_disk("./training_dataset/dev.spacy")